# Train and Apply Models

In [1]:
from ML.model_training import (
    omit_patient_video,
    single_user_split,
    train_lstm,
    train_random_forest,
    build_lstm_sequences
)
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    classification_report,
)
import numpy as np
import pandas as pd
from ML import utils
import sys
import random
from itertools import product

In [2]:
# remove_list = [0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 14, 16, 17, 18, 19, 22]
remove_list = []

def balance(X, y, seed=5):
    c = y.value_counts()
    if c.get("high", 0) == c.get("low", 0):
        return X.reset_index(drop=True), y.reset_index(drop=True)
    maj = c.idxmax()
    m = c.min()
    keep = y[y != maj].index.union(y[y == maj].sample(m, random_state=seed).index)
    return X.loc[keep].reset_index(drop=True), y.loc[keep].reset_index(drop=True)

subjects = [1, 3, 12, 15]
# subjects = []
# for _ in range(5):
#     subjects.append(random.randint(0, 22))
# print(subjects)
subjects = [1]

## LSTM CV Optimizer

In [5]:


param_grid = {
    "lr": [0.0001],
    "epochs": [100],
    "units": [128, 256, 512],
    "batch_size": [128, 256, 512],
}

best_params = None
best_mean_acc = -float("inf")

print("Starting global hyperparameter search...\n")

for lr, epochs, units, batch_size in product(
    param_grid["lr"],
    param_grid["epochs"],
    param_grid["units"],
    param_grid["batch_size"],
):
    combo_accs = []
    for i in subjects:
        relabel = False
        while True:
            while True:
                X_train_df, X_test_df, arousal_train, arousal_test = omit_patient_video(
                    target="arousal",
                    selected_user=i,
                    trials=18,
                    # holdout_videos=[2, 10, 15],
                    exclude_users=remove_list,
                    filename="datasets/features_table.csv",
                    relabel=relabel,
                )

                features = utils.filter_features(
                    X_train_df.columns,
                    remove_bands=["gamma", "delta"],
                )
                features = [c for c in features if c in X_train_df.columns]

                # Build sequence-level data (trials = (patient, video))
                X_train_seq, y_train_seq = build_lstm_sequences(
                    X_train_df,
                    features,
                    target_col="arousal",
                    thresh=3.8,
                    fixed_T=15,
                )
                X_test_seq, y_test_seq = build_lstm_sequences(
                    X_test_df,
                    features,
                    target_col="arousal",
                    thresh=3.8,
                    fixed_T=15
                )

                # minority = (y_train_seq == 1.0).sum() < (y_train_seq == 0.0).sum()
                # maj_label = 0.0 if minority else 1.0
                # min_label = 1.0 - maj_label

                # idx_maj = np.where(y_train_seq == maj_label)[0]
                # idx_min = np.where(y_train_seq == min_label)[0]

                # if len(idx_min) == 0:
                #     # no samples of one class, redo split
                #     print("No minority class in this split, re-drawing...")
                #     continue

                # reps = int(np.ceil(len(idx_maj) / len(idx_min)))
                # idx_min_upsampled = np.tile(idx_min, reps)[: len(idx_maj)]

                # idx_balanced = np.concatenate([idx_maj, idx_min_upsampled])
                # np.random.shuffle(idx_balanced)

                # X_train_bal = X_train_seq[idx_balanced]
                # y_train_bal = y_train_seq[idx_balanced]

                print("y_train counts:", np.bincount(y_train_seq.astype(int)))
                print("y_test counts:", np.bincount(y_test_seq.astype(int)))

                break

            lstm, X_test_eval, y_test_eval = train_lstm(
                X_train_seq,
                X_test_seq,
                y_train_seq,
                y_test_seq,
                lr=lr,
                epochs=epochs,
                units=units,
                batch_size=batch_size,
                dropout=0.2,
                recurrent_dropout=0.1,
                bidirectional=True,
            )
            y_prob = lstm.predict(X_test_eval).ravel()
            arousal_pred = (y_prob >= 0.5).astype(int)

            acc = accuracy_score(y_test_eval, arousal_pred)
            print(acc)
            relabel = False
            break

        combo_accs.append(float(acc))

        print("\nConfusion Matrix (subject):")
        print(confusion_matrix(y_test_eval, arousal_pred))
    mean_acc = float(np.mean(combo_accs))
    print(
        f"Params lr={lr}, epochs={epochs}, units={units}, batch_size={batch_size} "
        f"-> mean acc across subjects = {mean_acc:.4f}"
    )

    if mean_acc > best_mean_acc:
        best_lstm = lstm
        best_mean_acc = mean_acc
        best_params = {
            "lr": lr,
            "epochs": epochs,
            "units": units,
            "batch_size": batch_size,
        }


print("\nBest universal hyperparameters:")
print(best_params)
print(f"Mean accuracy across subjects (tuning): {best_mean_acc:.4f}\n")

Starting global hyperparameter search...

Held-out patient: 1 | Held-out (patient, video) trials: [(1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17)] | Excluded users: []
y_train counts: [224 172]
y_test counts: [9 9]
X_train_arr shape: (396, 15, 452)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 526ms/step
0.5555555555555556

Confusion Matrix (subject):
[[1 8]
 [0 9]]
Params lr=0.0001, epochs=100, units=128, batch_size=128 -> mean acc across subjects = 0.5556
Held-out patient: 1 | Held-out (patient, video) trials: [(1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17)] | Excluded users: []
y_train counts: [224 172]
y_test counts: [9 9]
X_train_arr shape: (396, 15, 452)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 527ms/step
0.5555555555555556

Confusion Matrix (subject):
[[1 8]
 [0 9]]
Params lr=0.0001, epochs=100, units=128

### Fine-tune for User

In [ ]:
from tensorflow.keras.models import clone_model
from tensorflow.keras import optimizers
import tensorflow as tf
import numpy as np
import pandas as pd

# df = utils.read_table("datasets/features_table_psd_only.csv").reset_index(drop=True)

# df = df[~df["patient_index"].isin([12])].reset_index(drop=True)

# X = df.drop(
#     columns=["patient_index", "video_index", "arousal", "valence", "Unnamed: 0"],
#     errors="ignore",
# )
# X = X.sort_index(axis=1)

# arousal_target = df["arousal"].astype(float)

# features = utils.filter_features(X.columns, remove_bands=["gamma", "delta"])
# X = X.loc[:, features]

# print("Global X shape:", X.shape)

# y_global = (arousal_target > 3.8).astype("int32")

# best_lstm, X_test_eval, y_test_eval = train_lstm(
#     X,
#     None,
#     y_global,
#     None,
#     lr=0.0001,
#     epochs=25,
#     units=128,
#     batch_size=128,
#     dropout=0.5,
#     recurrent_dropout=0.5,
#     bidirectional=True,
#     bidirectional=True,
# )

acc_list = []

for i in range(0, 18):
    X_user_train, X_user_test, arousal_user_train, arousal_user_test = (
        single_user_split(
            target="arousal",
            selected_user=12,
            k_holdouts=1,
            # holdout_videos=[2, 7, 10, 15, 17],
            filename="datasets/features_table.csv",
        )
    )

    arousal_user_train = pd.Series(arousal_user_train, dtype="float32")
    arousal_user_test = pd.Series(arousal_user_test, dtype="float32")

    y_user_train = (arousal_user_train > 3.8).astype("int32").to_numpy()
    y_user_test = (arousal_user_test > 3.8).astype("int32").to_numpy()

    X_user_train = X_user_train.sort_index(axis=1)
    X_user_test = X_user_test.sort_index(axis=1)

    X_user_train = X_user_train.loc[:, features]
    X_user_test = X_user_test.loc[:, features]

    X_user_train = X_user_train.to_numpy(dtype="float32")
    X_user_test = X_user_test.to_numpy(dtype="float32")

    X_user_train = X_user_train.reshape(-1, 1, X_user_train.shape[1])
    X_user_test = X_user_test.reshape(-1, 1, X_user_test.shape[1])

    print("User train shape:", X_user_train.shape)
    print("User test shape:", X_user_test.shape)

    user_model = clone_model(best_lstm)
    user_model.build(best_lstm.input_shape)
    user_model.set_weights(best_lstm.get_weights())

    for layer in user_model.layers[:]:
        layer.trainable = False

    user_model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    history = user_model.fit(
        X_user_train,
        y_user_train,
        validation_split=0.2,
        verbose=False,
        batch_size=64,
        epochs=10,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(
                patience=3, restore_best_weights=True, monitor="val_loss"
            )
        ],
    )

    test_loss, test_acc = user_model.evaluate(X_user_test, y_user_test, verbose=0)
    print(f"\nRun {i} - User Test accuracy: {test_acc:.4f}")

    acc_list.append(test_acc)

    y_prob = user_model.predict(X_user_test).ravel()
    y_pred = (y_prob >= 0.5).astype("int32")

    cm = confusion_matrix(y_user_test, y_pred, labels=[0, 1])
    print("\nConfusion Matrix (rows=true, cols=pred):")
    print(cm)

    print("\nOverall accuracy:", accuracy_score(y_user_test, y_pred))

acc_array = np.array(acc_list)
print(f"\nMean user test accuracy over {len(acc_array)} runs: {acc_array.mean():.4f}")
print(f"Std of user test accuracy over {len(acc_array)} runs: {acc_array.std():.4f}")

User train shape: (1670, 1, 98)
User test shape: (173, 1, 98)


/Users/tolas/Documents/coding/Tetris-EEG/venv/lib/python3.10/site-packages/keras/src/backend/tensorflow/trainer.py:86: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")



Run 0 - User Test accuracy: 0.6590
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step

Confusion Matrix (rows=true, cols=pred):
[[  0   0]
 [ 59 114]]

Overall accuracy: 0.6589595375722543
User train shape: (1754, 1, 98)
User test shape: (89, 1, 98)


/Users/tolas/Documents/coding/Tetris-EEG/venv/lib/python3.10/site-packages/keras/src/backend/tensorflow/trainer.py:86: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")



Run 1 - User Test accuracy: 0.0787
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 270ms/step

Confusion Matrix (rows=true, cols=pred):
[[ 0  0]
 [82  7]]

Overall accuracy: 0.07865168539325842
User train shape: (1690, 1, 98)
User test shape: (153, 1, 98)


...


Run 15 - User Test accuracy: 0.0102
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 548ms/step

Confusion Matrix (rows=true, cols=pred):
[[ 1 97]
 [ 0  0]]

Overall accuracy: 0.01020408163265306
User train shape: (1749, 1, 98)
User test shape: (94, 1, 98)


/Users/tolas/Documents/coding/Tetris-EEG/venv/lib/python3.10/site-packages/keras/src/backend/tensorflow/trainer.py:86: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")



Run 16 - User Test accuracy: 0.2021
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 268ms/step

Confusion Matrix (rows=true, cols=pred):
[[19 75]
 [ 0  0]]

Overall accuracy: 0.20212765957446807
User train shape: (1660, 1, 98)
User test shape: (183, 1, 98)


/Users/tolas/Documents/coding/Tetris-EEG/venv/lib/python3.10/site-packages/keras/src/backend/tensorflow/trainer.py:86: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")



Run 17 - User Test accuracy: 0.2077
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step

Confusion Matrix (rows=true, cols=pred):
[[ 38 145]
 [  0   0]]

Overall accuracy: 0.20765027322404372

Mean user test accuracy over 18 runs: 0.4796
Std of user test accuracy over 18 runs: 0.3579
